In [ ]:
import os
import numpy as np
import cv2
import zipfile
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

 with zipfile.ZipFile('/content/leapGestRecog.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

 def load_images(directory):
    data = []
    labels = []
    for folder in os.listdir(directory):
        folder_path = os.path.join(directory, folder)
        if os.path.isdir(folder_path):
            for img in os.listdir(folder_path):
                img_path = os.path.join(folder_path, img)
                gesture_img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if gesture_img is not None:
                    try:
                        gesture_img = cv2.resize(gesture_img, (128, 128))
                        data.append(gesture_img)
                        labels.append(folder)
                    except Exception as e:
                        pass
    return np.array(data), np.array(labels)

data, labels = load_images('/content/leapGestRecog')

 data = data / 255.0
data = data.reshape(-1, 128, 128, 1)

 lb = LabelBinarizer()
labels = lb.fit_transform(labels)

 X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

 model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 1)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(lb.classes_), activation='softmax')
])

 optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

 history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

 test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_acc * 100:.2f}%")

 plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

 model.save('hand_gesture_cnn_model.h5')
import pickle
with open('label_binarizer.pkl', 'wb') as f:
    pickle.dump(lb, f)
